# Домашнее задание №4

Асланов А.Б., ИУ9-21М

**Алгоритм Роккьо** - использует векторное пространство, чтобы найти лучший запрос на основе пользовательской разметки.

In [1]:
import pandas as pd
import numpy as np

alpha = 0.7
beta = 0.3
documents_num = 1000000   # общее число документов
relevant_docs_vecs = 2   # множество известных релевантных doc-векторов

d = {'words':['отбор', 'кандидат', 'отобрать', 'претендент', 'выбрать'],
     'TF_d1':[0,1,1,1,0],
     'TF_d2':[1,0,0,1,1],
     'q':[1,1,0,0,0],
     'df':[70000, 70000, 30000, 50000, 70000]
     }

dataframe = pd.DataFrame(data=d)

TF-IDF считаем по формуле:

$tfidf_{d}=tf*idf=ln(TF_{d} + 1)*log_{10}(\frac{documents\_num}{df})$

In [2]:
dataframe['idf'] = np.log10(documents_num/dataframe['df'])
dataframe['d1_tf-idf'] = np.log(dataframe['TF_d1']+1)*dataframe['idf']
dataframe['d2_tf-idf'] = np.log(dataframe['TF_d2']+1)*dataframe['idf']

In [3]:
dataframe

,TF_d1,TF_d2,df,q,words,idf,d1_tf-idf,d2_tf-idf
0,0,1,70000,1,отбор,1.154902,0.000000,0.800517
1,1,0,70000,1,кандидат,1.154902,0.800517,0.000000
2,1,0,30000,0,отобрать,1.522879,1.055579,0.000000
3,1,1,50000,0,претендент,1.301030,0.901805,0.901805
4,0,1,70000,0,выбрать,1.154902,0.000000,0.800517


In [7]:
# Нормировка вектора tf-idf в диапазон [0; 1]
dataframe['d1_norm'] = dataframe['d1_tf-idf'] / np.sqrt(sum(dataframe['d1_tf-idf'] ** 2)) * np.sqrt(sum(dataframe['TF_d1']))
dataframe['d2_norm'] = dataframe['d2_tf-idf'] / np.sqrt(sum(dataframe['d2_tf-idf'] ** 2)) * np.sqrt(sum(dataframe['TF_d2']))
dataframe

,TF_d1,TF_d2,df,q,words,idf,d1_tf-idf,d2_tf-idf,d1_norm,d2_norm
0,0,1,70000,1,отбор,1.154902,0.000000,0.800517,0.000000,0.957963
1,1,0,70000,1,кандидат,1.154902,0.800517,0.000000,0.865179,0.000000
2,1,0,30000,0,отобрать,1.522879,1.055579,0.000000,1.140844,0.000000
3,1,1,50000,0,претендент,1.301030,0.901805,0.901805,0.974649,1.079173
4,0,1,70000,0,выбрать,1.154902,0.000000,0.800517,0.000000,0.957963


In [8]:
q_mod = alpha * dataframe['q'] + beta * 1/relevant_docs_vecs * (dataframe['d1_norm'] + dataframe['d2_norm'])

In [9]:
res = pd.DataFrame({'q_mod':q_mod, 'words':['отбор', 'кандидат', 'отобрать', 'претендент', 'выбрать']})
res

,q_mod,words
0,0.843694,отбор
1,0.829777,кандидат
2,0.171127,отобрать
3,0.308073,претендент
4,0.143694,выбрать


### Вывод

Алгоритм Роккьо описывает обратную связь по релевантности документов. В связи с тем, что слово "претендент" встретилось в обоих документах, результат выдал о повышении его вклада в модифицированном векторе запроса.